In [ ]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [ ]:
map = gpd.read_file("Sidewalk_width_crossings_small.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossings')

In [ ]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
edges = G.edges

In [ ]:
# print(nodes)

In [ ]:
#Pick random ones or pick manually that make sense - to experiment
S = (120548.6120283842, 486088.19577846595)
T = (120798.0893320718, 486128.7633437495)

In [ ]:
# Initialise the Gaussian process for 2 objectives
gp = gaussian_process.GPPairwise(num_objectives=2, std_noise=0.01, kernel_width=0.15,prior_mean_type='zero', seed=None)

In [ ]:
P = [] #Pareto set

# Path initialisation
for i in objectives:
    p = nx.shortest_path(G, source=S, target=T, weight=i, method='dijkstra') #Dijkstra algorithm
    P.append(p)
P

In [ ]:
C = min(P, key=lambda p: (p[0], p[1]))
C

In [ ]:
# User ranking: Compare paths in P
user_preference = utils_user.UserPreference(num_objectives=2, std_noise=0.1)

# input_domain = utils_ccs.get_pcs_grid(ccs_size=ccs_size, num_objectives=2, eucledian_dist=pcs_point_dist, min_size=min_size, seed=123)
# # Rescale on input domain
# user_preference.rescale_on_input_domain(input_domain)

In [ ]:
params = utils_parameters.get_parameter_dict(query_type='ranking', num_objectives=2,  utility_noise=0.01)
# experiment = utils_experiment.Experiment(params)
# result = experiment.run(recalculate=False)
# result
        # # -- true utility --
        #
        # # get the input domain
        # input_domain = result[1]

In [ ]:
add_noise = True  # Specify whether to add noise to the preferences
user_ranking = user_preference.ranking(C, add_noise) #Returns an array, sorted in descending order of utility
# user_ranking
# ranking = self.user.ranking(self.acquirer.history, add_noise=True)
#             self.dataset.add_ranked_preferences(ranking)
#             x_max = ranking[0]

In [ ]:
# Add the comparisons to GP
user_ranking_copy = np.copy(user_ranking)

gp.update(user_ranking_copy)


In [ ]:
p_star = max(P, key=lambda p: gp(p)) #path the user likes best and has the maximum a posteriori (MAP) estimate
p_star

In [ ]:
# Initialise the acquisition function
acq_fun = acquisition_function.DiscreteAcquirer(input_domain=C, query_type='ranking', seed=123, acquisition_type='expected improvement')


In [ ]:
while C:
    t = max(C, key=lambda v: acq_fun(v)) #Pick candidate target with highest value acquisition function

In [ ]:
    #TODO: remove t from C

